<a href="https://colab.research.google.com/github/yunmokoo/SPY-Forecast_MARS_LSTM/blob/main/SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install requests > make_outputrerquest.txt 2>&1
!pip install yfinance > make_outputyfinance.txt 2>&1
!pip install transformers > make_outputtransformers.txt 2>&1
!pip install xformers > make_outputxformers.txt 2>&1


In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz > wget_output.txt 2>&1
!tar -xzvf ta-lib-0.4.0-src.tar.gz > tar_output.txt 2>&1
%cd ta-lib
!./configure --prefix=/usr > config_output.txt 2>&1
!make > make_outputtalib.txt 2>&1
!make install > makeInstall_output.txt 2>&1
!pip install TA-Lib > pipInstall_output.txt 2>&1

/content/ta-lib


In [10]:
import requests
import json
import yfinance as yf
#import talib

import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import matplotlib as plt

from transformers import *
from transformers import GenerationMixin
from transformers import TFGenerationMixin
from transformers import FlaxGenerationMixin

from sklearn.preprocessing import MinMaxScaler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import DistilBertTokenizer, DistilBertModel

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError

From TA-lib...

1. MA                  All Moving Average
2. BBANDS              Bollinger Bands
3. MACD                Moving Average Convergence/Divergence
4. SMA                 Simple Moving Average
5. EMA                 Exponential Moving Average

In [11]:

data_SPY = yf.download("SPY", start="2001-01-01", end="2023-01-01")
df = pd.DataFrame(data_SPY)

# Prepare the input and target data
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
target_column = 'Close'

X = df[features].values
y = df[target_column].values.reshape(-1, 1)

# Split the data into train, test, and cross-validation sets (not shuffled due to sequential data)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Normalize the data for better training performance
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_val = (X_val - mean) / std
X_test = (X_test - mean) / std


[*********************100%***********************]  1 of 1 completed


In [18]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_model, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dense1 = tf.keras.layers.Dense(d_model, activation='relu')
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.dense2 = tf.keras.layers.Dense(d_model, activation='relu')
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=True):
        # Create a mask to handle variable-length sequences
        mask = tf.math.not_equal(inputs[:, :, 0], 0)

        # Add the attention mechanism
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        attention_output = self.dropout(attention_output, training=training)
        output1 = self.layer_norm1(inputs + attention_output)

        # Add the dense layer
        dense_output = self.dense1(output1)
        dense_output = self.dropout(dense_output, training=training)
        output2 = self.layer_norm2(output1 + dense_output)
        return output2


class TransformerModel(tf.keras.models.Model):
    def __init__(self, num_transformer_blocks, num_heads, d_model, num_dense_units, dropout_rate):
        super(TransformerModel, self).__init__()
        self.num_transformer_blocks = num_transformer_blocks
        self.transformer_blocks = [TransformerBlock(num_heads, d_model, dropout_rate) for _ in range(num_transformer_blocks)]
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(num_dense_units, activation='relu')
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=True):
        x = inputs
        for i in range(self.num_transformer_blocks):
            x = self.transformer_blocks[i](x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x, training=training)
        return self.dense2(x)


In [19]:
num_transformer_blocks = 2
num_heads = 4
d_model = 64
num_dense_units = 64
dropout_rate = 0.2
learning_rate = 1e-4
batch_size = 32
epochs = 10

# Build the model
model = TransformerModel(num_transformer_blocks, num_heads, d_model, num_dense_units, dropout_rate)

# Compile the model
optimizer = Adam(learning_rate)
loss = MeanSquaredError()
metrics = RootMeanSquaredError()
model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

# Evaluate the model on the test set
test_loss, test_rmse = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test RMSE:", test_rmse)

Epoch 1/10


ValueError: ignored

In [8]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

# Transformer


In [ ]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_normalized

,Open,High,Low,Close,Adj Close,Volume
0,0.155737,0.151608,0.147845,0.148199,0.079748,0.006263
1,0.146771,0.160983,0.148074,0.163306,0.089547,0.018587
2,0.162880,0.159688,0.161140,0.159758,0.087246,0.006818
3,0.159308,0.155190,0.151818,0.149115,0.080342,0.011073
4,0.150570,0.146806,0.148150,0.151556,0.081925,0.003829
...,...,...,...,...,...,...
5530,0.757896,0.763598,0.760294,0.768555,0.791514,0.065175
5531,0.765531,0.763818,0.764256,0.764868,0.787902,0.055703
5532,0.761981,0.764403,0.756358,0.753296,0.776566,0.077915
5533,0.757848,0.766745,0.762862,0.769849,0.792781,0.073373


In [ ]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

class SimpleTransformerModel(nn.Module):
    def __init__(self, num_classes):
        super(SimpleTransformerModel, self).__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = nn.Linear(self.encoder.config.hidden_size, num_classes)

    def forward(self, input_text):
        input_ids = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
        outputs = self.encoder(input_ids)
        last_hidden_state = outputs.last_hidden_state
        cls_token = last_hidden_state[:, 0, :]
        logits = self.fc(cls_token)
        return logits

# Assuming you have prepared your data as train_loader and have defined num_classes.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleTransformerModel(num_classes)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5  # Change this based on the number of epochs you want to run
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, batch_data in enumerate(train_loader):
        inputs, targets = batch_data
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_loss:.4f}")
